## Setup

In [1]:
import re
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.performance import *
from alphalens.plotting import *
from alphalens.tears import *

In [2]:
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')

### Version conflict

At the time of writing, `zipline` required `pandas<=0.22` so you may need to run `pip install -U zipline` to temporarily downgrade `pandas` or set up a separate environment.

In [ ]:
!pip install -U zipline

## Zipline AlphaFactor Test

We are first going to illustrate the zipline alpha factor research workflow in an offline environment. In particular, we will develop and test a simple mean-reversion factor that measures how much recent performance has deviated from the historical average. 

Short-term reversal is a common strategy that takes advantage of the weakly predictive pattern that stock price increases are likely to mean-revert back down over horizons from less than a minute to one month.

To this end, the factor computes the z-score for the last monthly return relative to the rolling monthly returns over the last year. At this point, we will not place any orders to simply illustrate the implementation of a CustomFactor and record the results during the simulation.

After some basic settings, `MeanReversion` subclasses `CustomFactor` and defines a `compute()` method. It creates default inputs of monthly returns over an also default year-long window so that the monthly_return variable will have 252 rows and one column for each security in the Quandl dataset on a given day.

The `compute_factors()` method creates a `MeanReversion` factor instance and creates long, short, and ranking pipeline columns. The former two contain Boolean values that could be used to place orders, and the latter reflects that overall ranking to evaluate the overall factor performance. Furthermore, it uses the built-in `AverageDollarVolume` factor to limit the computation to more liquid stocks

The result would allow us to place long and short orders. We will see in the next chapter how to build a portfolio by choosing a rebalancing period and adjusting portfolio holdings as new signals arrive.

- The `initialize()` method registers the compute_factors() pipeline, and the before_trading_start() method ensures the pipeline runs on a daily basis. 
- The `record()` function adds the pipeline's ranking column as well as the current asset prices to the performance DataFrame returned by the `run_algorithm()` function

We will use the factor and pricing data stored in the performance DataFrame to evaluate the factor performance for various holding periods in the next section, but first, we'll take a look at how to create more complex signals by combining several alpha factors from a diverse set of data sources on the Quantopian platform.

Run using jupyter notebook extension

In [3]:
%load_ext zipline

In [10]:
%%zipline --start 2015-1-1 --end 2018-1-1 --output single_factor.pickle

from zipline.api import (
    attach_pipeline,
    date_rules,
    time_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
    get_open_orders,
    calendars
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
import numpy as np
import pandas as pd

MONTH = 21
YEAR = 12 * MONTH
N_LONGS = N_SHORTS = 25
VOL_SCREEN = 1000


class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())


def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs': mean_reversion.bottom(N_LONGS),
                             'shorts': mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))


def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)


def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    record(factor_data=factor_data.ranking)

    assets = factor_data.index
    record(prices=data.current(assets, 'price'))

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = set(context.portfolio.positions.keys()) - set(longs.union(shorts))

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS)


def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)
    context.set_commission(commission.PerShare(cost=.01, min_trade_cost=0))
    context.set_slippage(slippage.VolumeShareSlippage())


def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2015-01-02 21:00:00+00:00,NaN,0.000000,NaN,-0.000535,NaN,NaN,0.000000e+00,1.000000e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0,NaN,1.000000e+07,0.000000e+00,0.000000e+00,1,[],0.0
2015-01-05 21:00:00+00:00,0.001658,-0.000148,0.001137,-0.018585,0.196709,0.008430,4.557735e+06,1.455774e+07,-4.559213e+06,-4.559213e+06,...,-4.559213e+06,-4.559213e+06,4,-11.224972,1.000000e+07,0.000000e+00,0.000000e+00,2,"[{u'commission': None, u'amount': -25524, u'si...",0.0
2015-01-06 21:00:00+00:00,0.041095,-0.004703,-0.369692,-0.027829,0.139099,0.010798,1.392954e+07,2.848728e+07,-1.853431e+07,-1.853431e+07,...,-1.853431e+07,-1.853431e+07,5,-9.457366,1.455774e+07,-4.559213e+06,-4.559213e+06,3,"[{u'commission': None, u'amount': -30574, u'si...",0.0
2015-01-07 21:00:00+00:00,0.092916,-0.017128,-1.422556,-0.015715,0.206972,-0.346724,5.821402e+06,3.430868e+07,-2.447996e+07,-2.447996e+07,...,-2.447996e+07,-2.447996e+07,5,-10.264901,2.848728e+07,-1.853431e+07,-1.853431e+07,4,"[{u'commission': None, u'amount': -22032, u'si...",0.0
2015-01-08 21:00:00+00:00,0.219120,-0.049569,-2.445576,0.001751,0.236040,-0.761751,4.882509e+06,3.919119e+07,-2.968687e+07,-2.968687e+07,...,-2.968687e+07,-2.968687e+07,5,-10.014855,3.430868e+07,-2.447996e+07,-2.447996e+07,5,"[{u'commission': None, u'amount': -50424, u'si...",0.0
2015-01-09 21:00:00+00:00,0.432739,-0.115691,-5.095987,-0.006276,0.218110,-0.269478,4.057730e+06,4.324892e+07,-3.440583e+07,-3.440583e+07,...,-3.440583e+07,-3.440583e+07,5,-9.932744,3.919119e+07,-2.968687e+07,-2.968687e+07,6,"[{u'commission': None, u'amount': -29366, u'si...",0.0
2015-01-12 21:00:00+00:00,0.650275,-0.057214,-2.310900,-0.014061,0.203321,-0.772985,1.669051e+06,4.491797e+07,-3.549011e+07,-3.549011e+07,...,-3.549011e+07,-3.549011e+07,5,-4.118403,4.324892e+07,-3.440583e+07,-3.440583e+07,7,"[{u'commission': None, u'amount': -19344, u'si...",0.0
2015-01-13 21:00:00+00:00,0.620539,-0.039174,-1.498151,-0.016834,0.188301,-0.792991,4.810981e+06,4.972895e+07,-4.012069e+07,-4.012069e+07,...,-4.012069e+07,-4.012069e+07,8,-2.478067,4.491797e+07,-3.549011e+07,-3.549011e+07,8,"[{u'commission': None, u'amount': -17806, u'si...",0.0
2015-01-14 21:00:00+00:00,0.684345,-0.109137,-3.344705,-0.022770,0.177393,-0.539288,8.473548e+05,5.057631e+07,-4.166767e+07,-4.166767e+07,...,-4.166767e+07,-4.166767e+07,8,-5.316664,4.972895e+07,-4.012069e+07,-4.012069e+07,9,"[{u'commission': None, u'amount': -47673, u'si...",0.0
2015-01-15 21:00:00+00:00,0.891547,-0.010570,-0.894328,-0.031722,0.170557,-1.225731,-9.093704e+05,4.966693e+07,-3.977263e+07,-3.977263e+07,...,-3.977263e+07,-3.977263e+07,7,0.156254,5.057631e+07,-4.166767e+07,-4.166767e+07,10,"[{u'commission': None, u'amount': -9372, u'sid...",0.0
